In [ ]:


#scp -i "/home/johan/Johan_key.pem" -o StrictHostKeyChecking=accept-new -r "am111_spark" ec2-user@ec2-184-72-157-243.compute-1.amazonaws.com:/home/ec2-user
#ssh -i "Johan_key.pem" ec2-user@ec2-3-82-37-175.compute-1.amazonaws.com

#ssh -i "Johan_key.pem" ec2-user@ec2-54-160-226-58.compute-1.amazonaws.com

#scp -i "/home/johan/Johan_key.pem" -r ec2-user@ec2-184-72-157-243.compute-1.amazonaws.com:/home/ec2-user/am111_spark/models/ ./ 


#could put this into bootstrap shell script or smth to transfer the data. Or just upload it directly.

#REMEBER TO USE ec2-user instead of root or ubuntu when using ubuntu


#so we ssh onto the emr, and then scp our shit over, then built the dockerfile, then run the dockerfile.


In [2]:
import boto3
import time
import os
import paramiko

In [3]:
#export AWS_PROFILE=personal
#FOR COMMAND LINE ARGUMENTS ON THIS PROFILE
#INSTANCE PROFILE FOR EMR IS emr-all
session = boto3.Session(profile_name='personal')

emr = session.client('emr', region_name='us-east-1')
# create s3 client
s3 = session.resource('s3', region_name='us-east-1')
# Create a cluster with the default configurations 

In [3]:
#checking if correct aws accnt by printing buckets
for bucket in s3.buckets.all():
    print(bucket.name)


aws-cloudtrail-logs-012609303642-cf34e868
aws-logs-012609303642-us-east-2
johan-spark-commoncrawl-logs


In [13]:
# createa a bucket for the logs
bucket = s3.create_bucket(Bucket='johan-spark-commoncrawl-logs')

In [7]:

# Create a cluster with the default configurations 
dockerami = 'TODO'
response = emr.run_job_flow(

    Name='train_fin_docker',
    LogUri='s3://commoncrawl-logs/',
    ReleaseLabel='emr-6.9.0',
    Instances={
        'InstanceGroups': [
            {
                'Name': 'Master nodes',
                'Market': 'ON_DEMAND',
                'InstanceRole': 'MASTER',
                'InstanceType': 'm5.4xlarge',
                'InstanceCount': 1,
                'EbsConfiguration': {
                    'EbsBlockDeviceConfigs': [
                        {
                            'VolumeSpecification': {
                                'SizeInGB': 128,
                                'VolumeType': 'gp2'
                            }
                        }
                    ]
                }
              },
            {
                'Name': 'worker nodes',
                'Market': 'ON_DEMAND',
                'InstanceRole': 'CORE',
                'InstanceType': 'm5.xlarge',
                'InstanceCount': 4,
            }
        ],
        'KeepJobFlowAliveWhenNoSteps': True,
        'TerminationProtected': False,
        'Ec2KeyName': 'Johan_key',
    },
    VisibleToAllUsers = True,
    JobFlowRole='EMR_EC2_DefaultRole',
    ServiceRole='EMR_DefaultRole',
    Applications=[
        {
            'Name': 'Spark'
        },
    ]        
,BootstrapActions=[ 
        {
            'Name': 'Packages setup',
            'ScriptBootstrapAction':
                {
                'Path': 's3://johan-spark-scripts/dockerbootstrap.sh'
                }
        }]
)



In [5]:
print(response)

{'JobFlowId': 'j-2DEYW1GT1NBDX', 'ClusterArn': 'arn:aws:elasticmapreduce:us-east-1:012609303642:cluster/j-2DEYW1GT1NBDX', 'ResponseMetadata': {'RequestId': 'dda82bd3-b845-43c1-9379-121cf5c5c868', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'dda82bd3-b845-43c1-9379-121cf5c5c868', 'content-type': 'application/x-amz-json-1.1', 'content-length': '118', 'date': 'Tue, 13 Dec 2022 19:15:33 GMT'}, 'RetryAttempts': 0}}


In [6]:
#get cluster id
cluster_id = response['JobFlowId']
#get cluster status
cluster_status = emr.describe_cluster(ClusterId=cluster_id)['Cluster']['Status']['State']
print(cluster_status)

STARTING


In [47]:
#use cluster id to get public dns
clusternoton = False
while clusternoton == False:
  emr_list_instance_rep = emr.list_instances(
          ClusterId=cluster_id,
          InstanceGroupTypes=[
              'MASTER',
          ],
          InstanceStates=[
              'RUNNING',
          ]
      )
  try:
    print(emr_list_instance_rep['Instances'][0]['PrivateIpAddress'])
    clusternoton = True
  except:
    pass

172.31.27.88


In [31]:
#add step to cluster
# need to first upload stuff to the cluster, which can be done with scp.
#then chmod the script?\
#usr/bin/python3: can't open file '/home/ec2-user/am111_spark/training_fin_classfier.py': [Errno 13] Permission denied
#this doesnt work, why? will do it manually for now.

#ignore this code, just do it manually this is bugged

response = emr.add_job_flow_steps(
    JobFlowId=cluster_id,
    Steps=[
        {
            'Name': 'bootstrap',
            'ActionOnFailure': 'CONTINUE',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': [
                    'sudo',
                    'pip3',
                    'install',
                    '-r',
                    '/home/ec2-user/am111_spark/requirements.txt',
                ]
                 }
        },
        {
            'Name': 'spark job',
            'ActionOnFailure': 'CONTINUE',
            'HadoopJarStep': {
                'Jar': 'command-runner.jar',
                'Args': [
                    'python',
                    '/home/ec2-user/am111_spark/training_fin_classfier.py',

                ]
            }
        }
    ]
)

In [7]:
#terminate the cluster
emr.terminate_job_flows(JobFlowIds=[cluster_id])

{'ResponseMetadata': {'RequestId': '1bebfcf8-42db-41a5-8775-062eefec44d5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1bebfcf8-42db-41a5-8775-062eefec44d5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 10 Dec 2022 21:28:51 GMT'},
  'RetryAttempts': 0}}

In [ ]:
#check cluster status every 20 seconds and print
cluster_status = emr.describe_cluster(ClusterId=cluster_id)['Cluster']['Status']['State']
while cluster_status != 'TERMINATED' or cluster_status != 'TERMINATED_WITH_ERRORS':
    cluster_status = emr.describe_cluster(ClusterId=cluster_id)['Cluster']['Status']['State']
    print(cluster_status)
    time.sleep(20)
